# Clustergrammer2 Viz Stoeckius 2017

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.a

['GSE100866_CD8_merged-RNA_umi.csv']


In [2]:
from clustergrammer2 import net
df = {}

>> clustergrammer2 backend version 0.2.14


In [3]:
def drop_ribo_mito(df):
    all_genes = df.index.tolist()
    print(len(all_genes))
    keep_genes = [x for x in all_genes if 'RPL' not in x]
    keep_genes = [x for x in keep_genes if 'RPS' not in x]
    print(len(keep_genes))

    df = df.loc[keep_genes]
    df.shape

    # Removing Mitochondrial Genes
    list_mito_genes = ['MTRNR2L11', 'MTRF1', 'MTRNR2L12', 'MTRNR2L13', 'MTRF1L', 'MTRNR2L6', 'MTRNR2L7',
                    'MTRNR2L10', 'MTRNR2L8', 'MTRNR2L5', 'MTRNR2L1', 'MTRNR2L3', 'MTRNR2L4']

    all_genes = df.index.tolist()
    mito_genes = [x for x in all_genes if 'MT-' == x[:3] or 
                 x.split('_')[0] in list_mito_genes]
    print(mito_genes)

    keep_genes = [x for x in all_genes if x not in mito_genes]
    df = df.loc[keep_genes]
    
    return df

In [4]:
def umi_norm(df):
    # umi norm
    barcode_umi_sum = df.sum()
    df_umi = df.div(barcode_umi_sum)
    return df_umi

In [5]:
df['ini'] = pd.read_csv('../input/GSE100866_CD8_merged-RNA_umi.csv', index_col=0)
new_cols = [(x, 'CD8 Status: ' + x.split('_')[1]) for x in df['ini'].columns.tolist()]
print(df['ini'].shape)
df['ini'].columns = new_cols
df['ini'].head()

(11757, 1774)


,"(CD8_off_GACCCCCCTTCT, CD8 Status: off)","(CD8_off_CCTGCATATCTT, CD8 Status: off)","(CD8_off_CCAAGATTGTGA, CD8 Status: off)","(CD8_off_CTCCGGACAACC, CD8 Status: off)","(CD8_off_CCACTTCCCTCT, CD8 Status: off)","(CD8_off_ACAATCCCGTGG, CD8 Status: off)","(CD8_off_ACCCTCAACCCT, CD8 Status: off)","(CD8_off_CAGGTGAATGTT, CD8 Status: off)","(CD8_off_GTCACGGTTGGC, CD8 Status: off)","(CD8_off_CGCCGCAACGAT, CD8 Status: off)","(CD8_off_GGTCCCTGTGTT, CD8 Status: off)","(CD8_off_GACGAAATCGCA, CD8 Status: off)","(CD8_off_GTGTCGCAATCC, CD8 Status: off)","(CD8_off_TAAGGATAATTT, CD8 Status: off)","(CD8_off_GTCGGGAGATTT, CD8 Status: off)","(CD8_off_GTCTGGGTTTCA, CD8 Status: off)","(CD8_off_CGCCCCTGTTTG, CD8 Status: off)","(CD8_off_GCACGGAAGTAT, CD8 Status: off)","(CD8_off_CCTAGTTCCTTG, CD8 Status: off)","(CD8_off_ACTTAATGACAA, CD8 Status: off)","(CD8_off_CGCCGTGAACGC, CD8 Status: off)","(CD8_off_TCTGTGTCTTGA, CD8 Status: off)","(CD8_off_GACAAGCAAATA, CD8 Status: off)","(CD8_off_ATGGAGGGTAGA, CD8 Status: off)","(CD8_off_TCTAGGACTCTA, CD8 Status: off)","(CD8_off_TCTACCGCGGGG, CD8 Status: off)","(CD8_off_GCCAACCTAGGA, CD8 Status: off)","(CD8_off_ACGCGCGTCGGG, CD8 Status: off)","(CD8_off_TACACCGCCTGG, CD8 Status: off)","(CD8_off_CAGATGCGAAAT, CD8 Status: off)","(CD8_off_ATCACCCATTGT, CD8 Status: off)","(CD8_off_GATAAGGCTCTC, CD8 Status: off)","(CD8_off_CATAGTCACAGG, CD8 Status: off)","(CD8_off_CTGGTGATAGCA, CD8 Status: off)","(CD8_off_CAATTTGACCCG, CD8 Status: off)","(CD8_off_AGCTCGGCCTTA, CD8 Status: off)","(CD8_off_TGTTACACATTC, CD8 Status: off)","(CD8_off_CGCATTTTATGG, CD8 Status: off)","(CD8_off_GTAAAGCTACGT, CD8 Status: off)","(CD8_off_CGCAGCTCAGGT, CD8 Status: off)",...,"(CD8_high_TTTCGTACTACT, CD8 Status: high)","(CD8_high_TTATGTTGAAAG, CD8 Status: high)","(CD8_high_TCTTTAGGCATA, CD8 Status: high)","(CD8_high_GAACCCCGTAAA, CD8 Status: high)","(CD8_high_TAAGACGACTCA, CD8 Status: high)","(CD8_high_GTTTCTTATTAA, CD8 Status: high)","(CD8_high_TCACCTCGCCAG, CD8 Status: high)","(CD8_high_CAACATGTTTAG, CD8 Status: high)","(CD8_high_CTCATTGATGCG, CD8 Status: high)","(CD8_high_TAGGAAGTCTCG, CD8 Status: high)","(CD8_high_TCTCAGTAAGAG, CD8 Status: high)","(CD8_high_TTGAACCGTACG, CD8 Status: high)","(CD8_high_AAGGGGGTTTAT, CD8 Status: high)","(CD8_high_GAGATTGTACTC, CD8 Status: high)","(CD8_high_TTATAATGCAAC, CD8 Status: high)","(CD8_high_AGGGAGCGGCAA, CD8 Status: high)","(CD8_high_GGAGATGCGGAG, CD8 Status: high)","(CD8_high_AAAATCGTCCCT, CD8 Status: high)","(CD8_high_GGATCTGTGAGG, CD8 Status: high)","(CD8_high_GATTACAACTCA, CD8 Status: high)","(CD8_high_TATCAAAGTTAA, CD8 Status: high)","(CD8_high_TAGCTTGAAAAA, CD8 Status: high)","(CD8_high_ATTTCTTTGATA, CD8 Status: high)","(CD8_high_ATAACCTATACT, CD8 Status: high)","(CD8_high_TCATTCTGAGAA, CD8 Status: high)","(CD8_high_TCGTTCGTACCC, CD8 Status: high)","(CD8_high_TAGATATAAGTG, CD8 Status: high)","(CD8_high_TTTTTAGTCAAC, CD8 Status: high)","(CD8_high_AATAAAAGAAAT, CD8 Status: high)","(CD8_high_ACGATGCAAGTG, CD8 Status: high)","(CD8_high_ACGACTTGCTGG, CD8 Status: high)","(CD8_high_ACTCGTACCTTT, CD8 Status: high)","(CD8_high_ACCAACGCATCC, CD8 Status: high)","(CD8_high_GCACCGCCCTTC, CD8 Status: high)","(CD8_high_ACTCTCTAGCCA, CD8 Status: high)","(CD8_high_ACTTGGGAGTCC, CD8 Status: high)","(CD8_high_ACTCTGACACCA, CD8 Status: high)","(CD8_high_ACGCGCCTGATC, CD8 Status: high)","(CD8_high_GCCACACCTGTG, CD8 Status: high)","(CD8_high_ACTAAATCGTGC, CD8 Status: high)"
A2M-AS1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AACS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AAGAB,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AAK1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0

In [ ]:
df['ash'] = np.arcsinh(df['ini']/5)
df['ash-umi'] = umi_norm(drop_ribo_mito(df['ash']))
df['ash-umi'].shape

In [ ]:
ser_var = df['ash-umi'].var(axis=1).sort_values(ascending=False)
keep_genes = ser_var.index.tolist()[:1000]

df['ash-umi-var'] = df['ash-umi'].loc[keep_genes]
df['ash-umi-var'].shape

# Visualize Single Cell Data

In [ ]:
net.load_df(df['ash-umi-var'])
net.normalize(axis='row', norm_type='zscore')
net.clip(-5,5)
net.load_df(net.export_df().round(2))
net.widget()

Cells Cluster Based on CD8 Status